## 0. Load Packages

In [1]:
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
from skimage import io,color
import matplotlib.pyplot as plt
from os import walk
import os
from os.path import isfile, join
import imutils
import cv2
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


## 1. Model

### 1.1 Define Paths

In [2]:
path="images/60-80"
model_path=path+"/saved model/"

### 1.2 Some Functions

#### a. Image Functions

In [10]:
#Convert RGB images to LAB using CIELAB, then return input X (L), target Y (A. B)
def rgbtolab_batch(path):
    X=[]
    Y=[]
    j=1
    row=[]
    column=[]
    image = []
    files = [f for f in os.listdir(path) if isfile(join(path, f)) and f.endswith(".jpg")==True]
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
    for i in range(len(files)):
        filename=path + files[i]
        image = img_to_array(load_img(filename))
        image = np.array(image, dtype=float)
        column.append(len(image[0]))
        row.append(len(image))
        image = cv2.resize(image, (80,60))
        x = rgb2lab(1.0/255*image)[:,:,0]
        y = rgb2lab(1.0/255*image)[:,:,1:]
        y /= 128
        X.append(x)
        Y.append(y)
        j +=1
    return X,Y,row,column

#Convert LAB channels to RGB for a list of image
def labtorgb_batch(l,a,b,row,column):
    rgb=[]
    for i in range(len(l)):
        lab=np.dstack([l[i],a[i],b[i]])
        img=color.lab2rgb(lab)
        img = cv2.resize(img, (column[i],row[i]))
        rgb.append(img)
    return rgb

#Save colorized image
def savePredict(img_list,save_path):
    for i in range(len(img_list)):
        plt.imsave(("{}frame{}.jpg").format(save_path,i),img_list[i])

#### b. Load Pre-trained Model and Image

In [4]:
#Load Model and weights
def load_model(model_name):
    json_file = open('{}.json'.format(model_name), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    #load weights into new model
    loaded_model.load_weights("{}.h5".format(model_name))
    print("Loaded model from disk")
    return loaded_model

#Load Image
def load_image(image_path):
    X,Y,row,column = rgbtolab_batch(image_path)
    n_test=len(X)
    # Pre-process the data by reshaping
    X=np.array(X)
    Y=np.array(Y)
    X = X.reshape(n_test, 60, 80, 1)
    Y = Y.reshape(n_test, 60, 80, 2)
    return X,Y,row,column

#Load Model and Image - Colorization
def load_all(model_name,image_path):
    loaded_model=load_model(model_name)
    X,Y,row,column=load_image(image_path)
    n_test=len(X)
    loaded_model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])
    loaded_model.evaluate(X,Y,batch_size=10)
    Y_predict=loaded_model.predict(X)
    Y_predict *=128
    Y_a=Y_predict[:,:,:,0].reshape(n_test,60,80,1)
    Y_b=Y_predict[:,:,:,1].reshape(n_test,60,80,1)
    Y_ori_a=Y[:,:,:,0].reshape(n_test,60,80,1)*128
    Y_ori_b=Y[:,:,:,1].reshape(n_test,60,80,1)*128
    rgb_list=labtorgb_batch(X,Y_a,Y_b,row,column)
    rgb_original=labtorgb_batch(X,Y_ori_a,Y_ori_b,row,column)
    return rgb_list,rgb_original

## 2. Colorizing Video

### 2.1. Video Paths

In [5]:
#VIDEO="Roman Holiday Short thumnail.mp4"
VIDEO="blue sky.mp4"
bw_path = "video/b&w_video_frames/"
save_path = "video/colorized_video_frames/"
video_path = "video/colorized_video/"

### 2.2 Read video and convert to frames

In [6]:
video = "video/"+VIDEO
width = 80
vs = cv2.VideoCapture(video)

In [7]:
# loop over frames from the video stream
count = 0
success = True
while success:
	# grab the next frame and handle if we are reading from either
	# VideoCapture or VideoStream
	success, frame = vs.read()

	# if we are viewing a video and we did not grab a frame then we
	# have reached the end of the video
	if frame is None:
		break

	# resize the input frame, scale the pixel intensities to the
	frame = imutils.resize(frame, 80)
	frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)

	# resize the frame to 80x60 (the dimensions of the  model
	resized = cv2.resize(frame, (80, 60))

	# show the original frames
	cv2.imshow("Original", resized)

    # save the black and white frames
	cv2.imwrite(bw_path+"frame{}.jpg".format(str(count)), resized)
	count += 1
	key = cv2.waitKey(1) & 0xFF

	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break

vs.release()

# close any open windows
cv2.destroyAllWindows()

### 2.3 Load model and colorize the video

In [11]:
# Load model on generated frames, colorize the frames and save colorized frames
colorized, original = load_all(model_path+"model_0215",bw_path)
savePredict(colorized,save_path)

Loaded model from disk
2105/2105 [==============================] - 214s 101ms/step


### 2.4 Combine frames to a video file

In [14]:
### a. Define function to create a video from frames
def convert_frames_to_video(pathIn, pathOut, fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f)) and f.endswith(".jpg")==True] #If it's a valid path and exclude .DS_Store

    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
 
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)

    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'MJPG'), fps, size)

    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

In [15]:
### b. Generate the video file
pathIn= save_path
pathOut = video_path+'video.avi'
fps = 30.0
convert_frames_to_video(pathIn, pathOut, fps)
print("Colorized video is saved at {}".format(pathOut))

video/colorized_video_frames/frame0.jpg
video/colorized_video_frames/frame1.jpg
video/colorized_video_frames/frame2.jpg
video/colorized_video_frames/frame3.jpg
video/colorized_video_frames/frame4.jpg
video/colorized_video_frames/frame5.jpg
video/colorized_video_frames/frame6.jpg
video/colorized_video_frames/frame7.jpg
video/colorized_video_frames/frame8.jpg
video/colorized_video_frames/frame9.jpg
video/colorized_video_frames/frame10.jpg
video/colorized_video_frames/frame11.jpg
video/colorized_video_frames/frame12.jpg
video/colorized_video_frames/frame13.jpg
video/colorized_video_frames/frame14.jpg
video/colorized_video_frames/frame15.jpg
video/colorized_video_frames/frame16.jpg
video/colorized_video_frames/frame17.jpg
video/colorized_video_frames/frame18.jpg
video/colorized_video_frames/frame19.jpg
video/colorized_video_frames/frame20.jpg
video/colorized_video_frames/frame21.jpg
video/colorized_video_frames/frame22.jpg
video/colorized_video_frames/frame23.jpg
video/colorized_video_fram

video/colorized_video_frames/frame264.jpg
video/colorized_video_frames/frame265.jpg
video/colorized_video_frames/frame266.jpg
video/colorized_video_frames/frame267.jpg
video/colorized_video_frames/frame268.jpg
video/colorized_video_frames/frame269.jpg
video/colorized_video_frames/frame270.jpg
video/colorized_video_frames/frame271.jpg
video/colorized_video_frames/frame272.jpg
video/colorized_video_frames/frame273.jpg
video/colorized_video_frames/frame274.jpg
video/colorized_video_frames/frame275.jpg
video/colorized_video_frames/frame276.jpg
video/colorized_video_frames/frame277.jpg
video/colorized_video_frames/frame278.jpg
video/colorized_video_frames/frame279.jpg
video/colorized_video_frames/frame280.jpg
video/colorized_video_frames/frame281.jpg
video/colorized_video_frames/frame282.jpg
video/colorized_video_frames/frame283.jpg
video/colorized_video_frames/frame284.jpg
video/colorized_video_frames/frame285.jpg
video/colorized_video_frames/frame286.jpg
video/colorized_video_frames/frame

video/colorized_video_frames/frame612.jpg
video/colorized_video_frames/frame613.jpg
video/colorized_video_frames/frame614.jpg
video/colorized_video_frames/frame615.jpg
video/colorized_video_frames/frame616.jpg
video/colorized_video_frames/frame617.jpg
video/colorized_video_frames/frame618.jpg
video/colorized_video_frames/frame619.jpg
video/colorized_video_frames/frame620.jpg
video/colorized_video_frames/frame621.jpg
video/colorized_video_frames/frame622.jpg
video/colorized_video_frames/frame623.jpg
video/colorized_video_frames/frame624.jpg
video/colorized_video_frames/frame625.jpg
video/colorized_video_frames/frame626.jpg
video/colorized_video_frames/frame627.jpg
video/colorized_video_frames/frame628.jpg
video/colorized_video_frames/frame629.jpg
video/colorized_video_frames/frame630.jpg
video/colorized_video_frames/frame631.jpg
video/colorized_video_frames/frame632.jpg
video/colorized_video_frames/frame633.jpg
video/colorized_video_frames/frame634.jpg
video/colorized_video_frames/frame

video/colorized_video_frames/frame821.jpg
video/colorized_video_frames/frame822.jpg
video/colorized_video_frames/frame823.jpg
video/colorized_video_frames/frame824.jpg
video/colorized_video_frames/frame825.jpg
video/colorized_video_frames/frame826.jpg
video/colorized_video_frames/frame827.jpg
video/colorized_video_frames/frame828.jpg
video/colorized_video_frames/frame829.jpg
video/colorized_video_frames/frame830.jpg
video/colorized_video_frames/frame831.jpg
video/colorized_video_frames/frame832.jpg
video/colorized_video_frames/frame833.jpg
video/colorized_video_frames/frame834.jpg
video/colorized_video_frames/frame835.jpg
video/colorized_video_frames/frame836.jpg
video/colorized_video_frames/frame837.jpg
video/colorized_video_frames/frame838.jpg
video/colorized_video_frames/frame839.jpg
video/colorized_video_frames/frame840.jpg
video/colorized_video_frames/frame841.jpg
video/colorized_video_frames/frame842.jpg
video/colorized_video_frames/frame843.jpg
video/colorized_video_frames/frame

video/colorized_video_frames/frame1140.jpg
video/colorized_video_frames/frame1141.jpg
video/colorized_video_frames/frame1142.jpg
video/colorized_video_frames/frame1143.jpg
video/colorized_video_frames/frame1144.jpg
video/colorized_video_frames/frame1145.jpg
video/colorized_video_frames/frame1146.jpg
video/colorized_video_frames/frame1147.jpg
video/colorized_video_frames/frame1148.jpg
video/colorized_video_frames/frame1149.jpg
video/colorized_video_frames/frame1150.jpg
video/colorized_video_frames/frame1151.jpg
video/colorized_video_frames/frame1152.jpg
video/colorized_video_frames/frame1153.jpg
video/colorized_video_frames/frame1154.jpg
video/colorized_video_frames/frame1155.jpg
video/colorized_video_frames/frame1156.jpg
video/colorized_video_frames/frame1157.jpg
video/colorized_video_frames/frame1158.jpg
video/colorized_video_frames/frame1159.jpg
video/colorized_video_frames/frame1160.jpg
video/colorized_video_frames/frame1161.jpg
video/colorized_video_frames/frame1162.jpg
video/color

video/colorized_video_frames/frame1345.jpg
video/colorized_video_frames/frame1346.jpg
video/colorized_video_frames/frame1347.jpg
video/colorized_video_frames/frame1348.jpg
video/colorized_video_frames/frame1349.jpg
video/colorized_video_frames/frame1350.jpg
video/colorized_video_frames/frame1351.jpg
video/colorized_video_frames/frame1352.jpg
video/colorized_video_frames/frame1353.jpg
video/colorized_video_frames/frame1354.jpg
video/colorized_video_frames/frame1355.jpg
video/colorized_video_frames/frame1356.jpg
video/colorized_video_frames/frame1357.jpg
video/colorized_video_frames/frame1358.jpg
video/colorized_video_frames/frame1359.jpg
video/colorized_video_frames/frame1360.jpg
video/colorized_video_frames/frame1361.jpg
video/colorized_video_frames/frame1362.jpg
video/colorized_video_frames/frame1363.jpg
video/colorized_video_frames/frame1364.jpg
video/colorized_video_frames/frame1365.jpg
video/colorized_video_frames/frame1366.jpg
video/colorized_video_frames/frame1367.jpg
video/color

video/colorized_video_frames/frame1636.jpg
video/colorized_video_frames/frame1637.jpg
video/colorized_video_frames/frame1638.jpg
video/colorized_video_frames/frame1639.jpg
video/colorized_video_frames/frame1640.jpg
video/colorized_video_frames/frame1641.jpg
video/colorized_video_frames/frame1642.jpg
video/colorized_video_frames/frame1643.jpg
video/colorized_video_frames/frame1644.jpg
video/colorized_video_frames/frame1645.jpg
video/colorized_video_frames/frame1646.jpg
video/colorized_video_frames/frame1647.jpg
video/colorized_video_frames/frame1648.jpg
video/colorized_video_frames/frame1649.jpg
video/colorized_video_frames/frame1650.jpg
video/colorized_video_frames/frame1651.jpg
video/colorized_video_frames/frame1652.jpg
video/colorized_video_frames/frame1653.jpg
video/colorized_video_frames/frame1654.jpg
video/colorized_video_frames/frame1655.jpg
video/colorized_video_frames/frame1656.jpg
video/colorized_video_frames/frame1657.jpg
video/colorized_video_frames/frame1658.jpg
video/color

video/colorized_video_frames/frame1869.jpg
video/colorized_video_frames/frame1870.jpg
video/colorized_video_frames/frame1871.jpg
video/colorized_video_frames/frame1872.jpg
video/colorized_video_frames/frame1873.jpg
video/colorized_video_frames/frame1874.jpg
video/colorized_video_frames/frame1875.jpg
video/colorized_video_frames/frame1876.jpg
video/colorized_video_frames/frame1877.jpg
video/colorized_video_frames/frame1878.jpg
video/colorized_video_frames/frame1879.jpg
video/colorized_video_frames/frame1880.jpg
video/colorized_video_frames/frame1881.jpg
video/colorized_video_frames/frame1882.jpg
video/colorized_video_frames/frame1883.jpg
video/colorized_video_frames/frame1884.jpg
video/colorized_video_frames/frame1885.jpg
video/colorized_video_frames/frame1886.jpg
video/colorized_video_frames/frame1887.jpg
video/colorized_video_frames/frame1888.jpg
video/colorized_video_frames/frame1889.jpg
video/colorized_video_frames/frame1890.jpg
video/colorized_video_frames/frame1891.jpg
video/color

Colorized video is saved at video/colorized_video/video.avi
